**Alisa Todorova**
I am working on MacBook with a newer software version, and SageMath couldn't be downloaded on it. So I did the assignment both on SageMathCell and in Python, just so I can double-check the outputs better.

In [ ]:
# Task 1 with SageMathCell
def keyGen(n=512):
    "Generates an RSA key, where input n is the bitsize n of the RSA modulus N."

    # Generate two large distinct primes p and q of same bit-size k/2, where k is a parameter.
    p = random_prime(2^n)
    q = random_prime(2^n)

    Nn = p*q

    #Select a random integer e such that gcd(e, φ) = 1
    e=2^16+1
    gcd(e,(p-1)*(q-1))

    #Compute the unique integer d such that e · d ≡ 1 (mod φ)
    d = inverse_mod(e,(p-1)*(q-1))

    return Nn,p,q,e,d

keyGen()

In [ ]:
# Task 1 in python
import sympy
import random
import math

def keyGen(n=512):
    "Generates an RSA key, where input n is the bitsize n of the RSA modulus N."

    # Generate two large distinct primes p and q of same bit-size k/2, where k is a parameter.
    p = sympy.randprime(2**(n-1)+1, 2**n-1)
    q = sympy.randprime(2**(n-1)+1, 2**n-1)
    # print(f"p:{p}")
    # print(f"q:{q}")
    # Smaller numbers for testing:
    # p = 17
    # q = 11

    Nn = p*q
    # print(f"Nn:{Nn}")
    phi = (p-1)*(q-1)
    # print(f"phi:{phi}")

    #Select a random integer e such that gcd(e, φ) = 1
    e = random.randint(2, phi-1)
    while math.gcd(e, phi) != 1:
        e = random.randint(2, phi-1)
    # print(f"e:{e}")
    # Smaller number for testing
    # e = 3

    #Compute the unique integer d such that e · d ≡ 1 (mod φ)
    d = pow(e, phi-1, phi)
    # print(f"d:{d}")

    return Nn,p,q,e,d

key = keyGen()
print(key)

In [ ]:
# Task 2 with SageMathCell
def encrypt(m,Nn,e):
    c = m.powermod(e,Nn) #c = m^e mod n
    return c

def decrypt(c,Nn,d):
    m = c.powermod(d,Nn) #m = c^d mod n
    return m

def checkEnc():
    Nn,p,q,e,d=keyGen()
    m=ZZ.random_element(Nn)
    assert(decrypt(encrypt(m,Nn,e),Nn,d)==m)

checkEnc()

In [ ]:
# Task 2 in python
def encrypt(m,Nn,e):
    c = (m**e) % Nn #c = m^e mod n
    return c

def decrypt(c,Nn,d):
    m = (c**d) % Nn #m = c^d mod n
    return m

def checkEnc():
    Nn,p,q,e,d=keyGen()
    m = 89
    c = encrypt(m,Nn,e)
    dec = decrypt(c,Nn,d)
    print(f"Encrypted message:{c}")
    print(f"Decrypted message:{dec}")

checkEnc()


**Task 3:**
Implement the RSA signature scheme with signature σ = H(m)^d mod N, where the output size of the hash function H is the same as the bit size of N, minus 4 bits. For this, we can concatenate the evaluation of a hash function h (for example, SHA-1), using an index for the message, truncating the last block:
H(m) = h(m∥0) ∥ h(m∥1)∥ · · · ∥ h(m∥k)

In [ ]:
# Task 3 with SageMathCell

import hashlib

def sha1(m):
    h=hashlib.sha1()
    h.update(m.encode("utf-8"))
    return h.hexdigest()


# lN is the length of the modulus in bits
def fullHash(m,lN):
    k=ceil(lN/160)
    hf="".join(sha1(m+str(i)) for i in range(k))
    hf=hf[:lN//4-1]
    return Integer(hf,base=16)


def sign(m,Nn,d):
    lN = Nn.nbits()
    mu = fullHash(m, lN)
    s = pow(mu, d, Nn) #σ = µ(m)^d mod N
    return s

def verify(s,m,Nn,e):
    lN = Nn.nbits()
    mu = fullHash(m, lN)
    m_check = pow(s, e, Nn) #m = s^e mod N
    return mu == m_check

def checkSig():
    Nn,p,q,e,d=keyGen()
    m="message"
    assert(verify(sign(m,Nn,d),m,Nn,e))

checkSig()

In [ ]:
# Task 3 in python

import hashlib

def sha1(m):
    h=hashlib.sha1()
    h.update(m.encode("utf-8"))
    return h.hexdigest()


# lN is the length of the modulus in bits
def fullHash(m,lN):
    k=math.ceil(lN/160)
    hf="".join(sha1(m+str(i)) for i in range(k))
    hf=hf[:lN//4-1]
    return int(hf,base=16)


def sign(m,Nn,d):
    lN = Nn.bit_length()
    mu = fullHash(m, lN)
    s = pow(mu, d, Nn) #σ = µ(m)^d mod N
    return s

def verify(s,m,Nn,e):
    lN = Nn.bit_length()
    mu = fullHash(m, lN)
    m_check = pow(s, e, Nn) #m = s^e mod N
    return mu == m_check

def checkSig():
    Nn,p,q,e,d=keyGen()
    m="message"
    s = sign(m,Nn,d)
    print(f"sign:{s}")
    v = verify(s,m,Nn,e)
    print(f"verify:{v}")
    assert(verify(sign(m,Nn,d),m,Nn,e))

checkSig()
